# Задание 1 (2 балла)

Напишите класс `MyDict`, который будет полностью повторять поведение обычного словаря, за исключением того, что при итерации мы должны получать и ключи, и значения.

**Модули использовать нельзя**

In [35]:
# Ваш код здесь
class MyDict(dict):
    def __iter__(self):
        for key in super().__iter__():
            yield key, self[key]

In [36]:
dct = MyDict({"a": 1, "b": 2, "c": 3, "d": 25})
for key, value in dct:
    print(key, value)   

a 1
b 2
c 3
d 25


In [37]:
for key, value in dct.items():
    print(key, value)

a 1
b 2
c 3
d 25


In [38]:
for key in dct.keys():
    print(key)

a
b
c
d


In [39]:
dct["c"] + dct["d"]

28

# Задание 2 (2 балла)

Напишите функцию `iter_append`, которая "добавляет" новый элемент в конец итератора, возвращая итератор, который включает изначальные элементы и новый элемент. Итерироваться по итератору внутри функции нельзя, то есть вот такая штука не принимается
```python
def iter_append(iterator, item):
    lst = list(iterator) + [item]
    return iter(lst)
```

**Модули использовать нельзя**

In [40]:
def iter_append(iterator, item):
    for i in iterator:
        yield i
    yield item
    

my_iterator = iter([1, 2, 3])
new_iterator = iter_append(my_iterator, 4)

for element in new_iterator:
    print(element)
    break

1


# Задание 3 (5 баллов)

Представим, что мы установили себе некотурую библиотеку, которая содержит в себе два класса `MyString` и `MySet`, которые являются наследниками `str` и `set`, но также несут и дополнительные методы.

Проблема заключается в том, что библиотеку писали не очень аккуратные люди, поэтому получилось так, что некоторые методы возвращают не тот тип данных, который мы ожидаем. Например, `MyString().reverse()` возвращает объект класса `str`, хотя логичнее было бы ожидать объект класса `MyString`.

Найдите и реализуйте удобный способ сделать так, чтобы подобные методы возвращали экземпляр текущего класса, а не родительского. При этом **код методов изменять нельзя**

**+3 дополнительных балла** за реализацию того, чтобы **унаследованные от `str` и `set` методы** также возвращали объект интересующего нас класса (то есть `MyString.replace(..., ...)` должен возвращать `MyString`). **Переопределять методы нельзя**

**Модули использовать нельзя**

In [41]:
# Ваш код где угодно, но не внутри методов
def wrap_class(cls):
    
    def decor_function( attr):
        def wrap_result(*args, **kwargs):
            res = attr(*args, **kwargs)
            if isinstance(res, str):
                return MyString(res)
            if isinstance(res, set):
                return MySet(res)
            return res
        return wrap_result
    
    class CustomClassWrapper(cls):
        def __init__(self, *args, **kwargs):
            for attr_name in dir(self):
                if not attr_name.startswith('__') and not attr_name.endswith('__'):
                    attr = getattr(self, attr_name)
                    if callable(attr):
                        setattr(self, attr_name, decor_function(attr))
    return CustomClassWrapper

@wrap_class
class MyString(str):

    def reverse(self):
        return self[::-1]
    
    def make_uppercase(self):
        return "".join([chr(ord(char) - 32) if 97 <= ord(char) <= 122 else char for char in self])
    
    def make_lowercase(self):
        return "".join([chr(ord(char) + 32) if 65 <= ord(char) <= 90 else char for char in self])
    
    def capitalize_words(self):
        return " ".join([word.capitalize() for word in self.split()])

@wrap_class
class MySet(set):
    
    def is_empty(self):
        return len(self) == 0
    
    def has_duplicates(self):
        return len(self) != len(set(self))
    
    def union_with(self, other):
        return self.union(other)
    
    def intersection_with(self, other):
        return self.intersection(other)
    
    def difference_with(self, other):
        return self.difference(other)

In [42]:
string_example = MyString("Aa Bb Cc")
set_example_1 = MySet({1, 2, 3, 4})
set_example_2 = MySet({3, 4, 5, 6, 6})

print(type(string_example.reverse()))
print(type(string_example.make_uppercase()))
print(type(string_example.make_lowercase()))
print(type(string_example.capitalize_words()))
print(type(string_example.replace("Aa", "cc")))
print()
print(type(set_example_1.is_empty()))
print(type(set_example_2.has_duplicates()))
print(type(set_example_1.union_with(set_example_2)))
print(type(set_example_1.difference_with(set_example_2)))

<class '__main__.wrap_class.<locals>.CustomClassWrapper'>
<class '__main__.wrap_class.<locals>.CustomClassWrapper'>
<class '__main__.wrap_class.<locals>.CustomClassWrapper'>
<class '__main__.wrap_class.<locals>.CustomClassWrapper'>
<class '__main__.wrap_class.<locals>.CustomClassWrapper'>

<class 'bool'>
<class 'bool'>
<class '__main__.wrap_class.<locals>.CustomClassWrapper'>
<class '__main__.wrap_class.<locals>.CustomClassWrapper'>


# Задание 4 (5 баллов)

Напишите декоратор `switch_privacy`:
1. Делает все публичные **методы** класса приватными
2. Делает все приватные методы класса публичными
3. Dunder методы и защищённые методы остаются без изменений
4. Должен работать тестовый код ниже, в теле класса писать код нельзя

**Модули использовать нельзя**

In [43]:
def switch_privacy(cls):
    class SwitchedClass(cls):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            for attr_name in dir(self):
                attr = getattr(self, attr_name)
                
                if attr_name.startswith('__') and attr_name.endswith('__'):
                    continue
                if attr_name.startswith('_' + cls.__name__):
                    prefix_len = 1 + len(cls.__name__) + 2
                    setattr(self, attr_name[prefix_len:], attr)
                    continue
                if attr_name.startswith('_'):
                    continue
                setattr(self, f'_{cls.__name__}__{attr_name}', attr)

    return SwitchedClass

# Ваш код здесь
@switch_privacy
class ExampleClass:
    # Но не здесь
    def public_method(self):
        return 1
    
    def _protected_method(self):
        return 2
    
    def __private_method(self):
        return 3
    
    def __dunder_method__(self):
        pass

In [44]:
test_object = ExampleClass()
# print(dir(test_object))

test_object._ExampleClass__public_method()   # Публичный метод стал приватным

1

In [45]:
test_object.private_method()   # Приватный метод стал публичным

3

In [46]:
test_object._protected_method()   # Защищённый метод остался защищённым

2

In [47]:
test_object.__dunder_method__()   # Дандер метод не изменился

In [48]:
hasattr(test_object, "public_method"), hasattr(test_object, "private")   # Изначальные варианты изменённых методов не сохраняются

(True, False)

# Задание 5 (7 баллов)

Напишите [контекстный менеджер](https://docs.python.org/3/library/stdtypes.html#context-manager-types) `OpenFasta`

Контекстные менеджеры это специальные объекты, которые могут работать с конструкцией `with ... as ...:`. В них нет ничего сложного, для их реализации как обычно нужно только определить только пару dunder методов. Изучите этот вопрос самостоятельно

1. Объект должен работать как обычные файлы в питоне (наследоваться не надо, здесь лучше будет использовать **композицию**), но:
    + При итерации по объекту мы должны будем получать не строку из файла, а специальный объект `FastaRecord`. Он будет хранить в себе информацию о последовательности. Важно, **не строки, а именно последовательности**, в fasta файлах последовательность часто разбивают на много строк
    + Нужно написать методы `read_record` и `read_records`, которые по смыслу соответствуют `readline()` и `readlines()` в обычных файлах, но они должны выдавать не строки, а объект(ы) `FastaRecord`
2. Конструктор должен принимать один аргумент - **путь к файлу**
3. Класс должен эффективно распоряжаться памятью, с расчётом на работу с очень большими файлами
    
Объект `FastaRecord`. Это должен быть **датакласс** (см. про примеры декораторов в соответствующей лекции) с тремя полями:
+ `seq` - последовательность
+ `id_` - ID последовательности (это то, что в фаста файле в строке, которая начинается с `>` до первого пробела. Например, >**GTD326487.1** Species anonymous 24 chromosome) 
+ `description` - то, что осталось после ID (Например, >GTD326487.1 **Species anonymous 24 chromosome**)


Напишите демонстрацию работы кода с использованием всех написанных методов, обязательно добавьте файл с тестовыми данными в репозиторий (не обязательно большой)

**Можно использовать модули из стандартной библиотеки**

In [56]:
# Ваш код здесь
import os
from dataclasses import dataclass

@dataclass
class FastaRecord:
    seq: str
    id_: str
    description: str

class OpenFasta:
    def __init__(self, path):
        self.path = path
        self.__file = None
        
    def __enter__(self):
        self.open()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.__file = None
        return False
        
    def open(self):
        self.__file = open(self.path, 'r')
        
    def close(self):
        if self.__file is not None:
            self.__file.close()
            self.__file = None
            
    def read_record(self):
        if self.__file is None:
            raise IOError('Fasta file was not opened')
        id = None
        description = None
        seq = ""
        while True:
            pos = self.__file.tell()
            first_char = self.__file.read(1)
            self.__file.seek(pos)
            if first_char == '':
                if id is not None:
                    return FastaRecord(seq, id, description)
                else:
                    return None
            if first_char == '>':
                if id is not None:
                    return FastaRecord(seq, id, description)
            line = self.__file.readline()
            if line.startswith('>'):
                id, description = line[1::].split(' ', 1)
                description = description[:-1]
            else:
                seq += line[:-1]
            
    def read_records(self):
        while True:
            record = self.read_record()
            if record is None:
                break
            yield record
                

with OpenFasta(os.path.join("data", "example.fasta")) as fasta:
    record = fasta.read_record()
    print(f"first record: {record}")
    for r in fasta.read_records():
        print(f"other record: {r}")
        

first record: FastaRecord(seq='CTCTTTCTCGATTCAGTGGATGGTGGTGCATGGCCGTTCTTAGTTGGTGGAGTGATCTGTCTGGTTAATTCCGATAACGAACGAGACTCTCCCCTGCTAACTAGTCGGTTGGATCTTGATCAGTGGGTTGAGTGCCGAGTAACTCTCGCTGTGCGTGGTGTGCGTCTCGTTCCTCACGGTTCGTTGCGTGCTCATGTGTGGCGTTGGATGCCTTGGATCACACTGCACCCCAAGTCCGCCGCTTTGCTATAACTCTTCTTAGAGGGATTCACGGCACTGAAGCCGTAAGAGACTGAGAGCAATAACAGGTCTGTGATGCCCTTAGATGCTCTGGGCCGCACGCGCGCTACAATGCCGGGCACAGCAGGTGTTTCTCCTGCTCCGACAGGAGCGGGTAACCCTCTGAAAGCCAGTGTTCTTGCAAGTGCTTCCTCCCTCGTGGAGTTGCACAGAGGCGTGCTTGGGCCTGGGACTTGCAATTCTTTCCCACCAACGAGGAATGCCTAGTAGCGGCGGGTCACTGAGCCCGCCGTGACTCAGTCCCTGCCCTTTGTACACACCGCCCGTCGCTACTACCGATTGGGTGTACCAGTGAGAGCCTTGGACTGGCGTCCGTTGTTGCTCGCAAGAGTGACAAGCGGGTCCGACGGAAAGAGGTCCGAACTGGTGCACTTAGAGGAAGTAAA', id_='AY926868.1', description='Parapallasea borowskii 18S ribosomal RNA gene, partial sequence')
other record: FastaRecord(seq='CTGTGTTCACACGGCGAGACCGCGRACGGCTCATTAAATCAGTCGTGGTCCAAATGGGCCAGTGCAAATCCTACTTGGATAACTGTGGTAATTCCAGAGCTAATACATGCAACTGATATCCCGAAGGCGGTGTTTGTTCTTGCTTGCCTTGC

# Задание 6 (7 баллов)

1. Напишите код, который позволит получать все возможные (неуникальные) генотипы при скрещивании двух организмов. Это может быть функция или класс, что вам кажется более удобным.

Например, все возможные исходы скрещивания "Aabb" и "Aabb" (неуникальные) это

```
AAbb
AAbb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
aabb
aabb
```

2. Напишите функцию, которая вычисляет вероятность появления определённого генотипа (его ожидаемую долю в потомстве).
Например,

```python
get_offspting_genotype_probability(parent1="Aabb", parent2="Aabb", target_genotype="Аabb")   # 0.5

```

3. Напишите код, который выводит все уникальные генотипы при скрещивании `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` и `'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН'`, которые содержат в себе следующую комбинацию аллелей `'АаБбВвГгДдЕеЖжЗзИиЙйКкЛл'`
4. Напишите код, который расчитывает вероятность появления генотипа `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` при скрещивании `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн` и `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн`

Важные замечания:
1. Порядок следования аллелей в случае гетерозигот всегда должен быть следующим: сначала большая буква, затем маленькая (вариант `AaBb` допустим, но `aAbB` быть не должно)
2. Подзадачи 3 и 4 могут потребовать много вычислительного времени (до 15+ минут в зависимости от железа), поэтому убедитесь, что вы хорошо протестировали написанный вами код на малых данных перед выполнением этих задач. Если ваш код работает **дольше 20 мин**, то скорее всего ваше решение не оптимально, попытайтесь что-нибудь оптимизировать. Если оптимальное решение совсем не получается, то попробуйте из входных данных во всех заданиях убрать последний ген (это должно уменьшить время выполнения примерно в 4 раза), но **за такое решение будет снято 2 балла**
3. Несмотря на то, что подзадания 2, 3 и 4 возможно решить математически, не прибегая к непосредственному получению всех возможных генотипов, от вас требуется именно brute-force вариант алгоритма

**Можно использовать модули из стандартной библиотеки питона**, но **за выполнение задания без использования модулей придусмотрено +3 дополнительных балла**

In [57]:
# Ваш код здесь (1 и 2 подзадание)

In [58]:
# Ваш код здесь (3 подзадание)

In [59]:
# Ваш код здесь (4 подзадание)